In [3]:
import requests
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import os.path
from os import path
from pathlib import Path



In [4]:
#Compiling URLs to scrape
years = list(range(2000, 2023))
url_big12 = 'https://www.sports-reference.com/cbb/conferences/big-12/{}-stats.html'
url_sec = 'https://www.sports-reference.com/cbb/conferences/sec/{}-stats.html'
url_big10 = 'https://www.sports-reference.com/cbb/conferences/big-ten/{}-stats.html'
url_bigeast = 'https://www.sports-reference.com/cbb/conferences/big-east/{}-stats.html'
url_pac12 = 'https://www.sports-reference.com/cbb/conferences/pac-12/{}-stats.html'
url_acc = 'https://www.sports-reference.com/cbb/conferences/acc/{}-stats.html'
url_aac = 'https://www.sports-reference.com/cbb/conferences/aac/{}-stats.html'
url_mwc = 'https://www.sports-reference.com/cbb/conferences/mwc/{}-stats.html'
url_wcc = 'https://www.sports-reference.com/cbb/conferences/wcc/{}-stats.html'
url_a10 = 'https://www.sports-reference.com/cbb/conferences/atlantic-10/{}-stats.html'
url_mvc = 'https://www.sports-reference.com/cbb/conferences/mvc/{}-stats.html'
url_cusa = 'https://www.sports-reference.com/cbb/conferences/cusa/{}-stats.html'
url_col = 'https://www.sports-reference.com/cbb/conferences/colonial/{}-stats.html'
url_south = 'https://www.sports-reference.com/cbb/conferences/southern/{}-stats.html'
url_wac = 'https://www.sports-reference.com/cbb/conferences/wac/{}-stats.html'
url_maac = 'https://www.sports-reference.com/cbb/conferences/maac/{}-stats.html'
url_bw = 'https://www.sports-reference.com/cbb/conferences/big-west/{}-stats.html'
url_sb = 'https://www.sports-reference.com/cbb/conferences/sun-belt/{}-stats.html'
url_ivy = 'https://www.sports-reference.com/cbb/conferences/ivy/{}-stats.html'
url_mac = 'https://www.sports-reference.com/cbb/conferences/mac/{}-stats.html'
url_as = 'https://www.sports-reference.com/cbb/conferences/atlantic-sun/{}-stats.html'
url_ovc = 'https://www.sports-reference.com/cbb/conferences/ovc/{}-stats.html'
url_summit = 'https://www.sports-reference.com/cbb/conferences/summit/{}-stats.html'
url_big_south = 'https://www.sports-reference.com/cbb/conferences/big-south/{}-stats.html'
url_big_sky = 'https://www.sports-reference.com/cbb/conferences/big-sky/{}-stats.html'
url_america_east = 'https://www.sports-reference.com/cbb/conferences/america-east/{}-stats.html'
url_horizon = 'https://www.sports-reference.com/cbb/conferences/horizon/{}-stats.html'
url_patriot = 'https://www.sports-reference.com/cbb/conferences/patriot/{}-stats.html'
url_southland = 'https://www.sports-reference.com/cbb/conferences/southland/{}-stats.html'
url_northeast = 'https://www.sports-reference.com/cbb/conferences/northeast/{}-stats.html'
url_meac = 'https://www.sports-reference.com/cbb/conferences/meac/{}-stats.html'
url_swac = 'https://www.sports-reference.com/cbb/conferences/swac/{}-stats.html'
url_draft =     'https://www.nbadraft.net/{}-nba-draft-combine-measurements/'
url_draft_summary = 'https://basketball.realgm.com/nba/draft/past_drafts/{}'

list_of_urls = [
url_big12, 
url_sec, 
url_big10,
url_bigeast,
url_pac12,
url_acc,
url_aac,
url_mwc,
url_wcc,
url_a10,
url_mvc,
url_cusa,
url_col,
url_south,
url_wac,
url_maac,
url_bw,
url_sb,
url_ivy,
url_mac,
url_as,
url_ovc ,
url_summit,
url_big_south,
url_big_sky,
url_america_east,
url_horizon,
url_patriot,
url_southland,
url_northeast,
url_meac,
url_swac
]
#Table ID
NCAA_id = 'conference-stats'

In [5]:
#Generates File name
def generate_name(url, year):
    end_index = url[49:].find('/')
    name = url[49:49+end_index]+'_'+str(year)
    return(name)
    
        
    

In [6]:
#Generates File Directory
def generate_directory(directory, file_name):
    generated_directory = directory + '/{}.html'.format(file_name)
    return(generated_directory)
    

In [7]:
#Downloads HTML File from given URL to directory
def get_html_request(directory, url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open(generate_directory(directory, generate_name(url, year)), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [8]:
#Downloads HTML File from given URL to /NCAA
def get_html_request_NCAA(url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open('NCAA/{}.html'.format(generate_name(url, year)), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [9]:
#Downloads HTML File from given URL to /Draft
def get_html_request_draft(url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open('Draft/{}.html'.format(year), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [10]:
#Downloads HTML File from given URL to /NCAA, uses Selenium to execute javascript if required
def get_html_api(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    for year in years:        
        temp_url = url.format(year)
        driver.get(temp_url)
        driver.execute_script("window.scrollTo(1,10000)")
        time.sleep(4)
        with open("NCAA/{}.html".format(generate_name(url, year)), "w+",encoding="utf-8") as f:
            f.write(driver.page_source)
    driver.quit()

In [11]:
#Searches webpage for all tables
def search_url_for_tables(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    return(tables)

In [12]:
#Uses requests to get html and searches for all tables, returning them as a df with first row as header
def get_df_from_url(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    df = pd.read_html(str(tables))[0]
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data unless the header row
    df.columns = new_header
    return(df)

In [13]:
#Uses requests to get html and searches for all tables, returning them as a df with no header
def get_df_from_url_noheader(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    df = pd.read_html(str(tables))[0]
    return(df)

In [14]:
#Opens html file and extracts table to df from table id
def html_to_df_from_tableid(file_name, table_id):
    with open('NCAA/{}.html'.format(file_name), encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        soup.find('tr', class_= 'over_header').decompose() #Removes 'over_header' element of table
        table = soup.find(id=table_id)
        df = pd.read_html(str(table))[0]
    return df

#Uses URL to get df based on table id
def html_to_df_from_tableid_no_download(url, table_id):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find(id=table_id)
    df = pd.read_html(str(table))[0]
    return df

#Uses URL to get df based on table class
def html_to_df_from_table_class_no_download(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find('table', {'class' : ["tablesaw", "tablesaw-swipe", "tablesaw-sortable"]})
    df = pd.read_html(str(table))[0]
    return df



In [15]:
#Removes Positons from player names as to only include player name in 'Player' Collumn
def clean_player_name(series):
    new_series = []
    for item in series:
        index = 0
        for char in reversed(item):
            if char.capitalize() == char or char == '-':
                index = index - 1
            else:
                break
        new_series.append(item[:index].strip())
    return(new_series)

In [16]:
#Converts from inches to cm so model can interpret data
def inch_to_cm(series):
    new_series = []
    for measurement in series:
        measurement = measurement.strip()
        feet = float(measurement[0])
        inches = float(measurement[2:-1])
        height_inches = feet*12 + inches
        height_cm = round(height_inches * 2.54, 2)
        new_series.append(height_cm)
    return(new_series)


In [17]:
#Removes instances in dataset where last character is a non-int value 
def remove_last_char(series):
    new_series = []
    for item in series:
        if isinstance(item, str):
            if item[-1] == '‘' or item[-1] == '′' or item[-1] == '%':
                new_series.append(item[:-1])
            else:
                new_series.append(item)
        else:
            new_series.append(item)
    return(new_series)    

In [18]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats
import time
from time import sleep

combine_dfs = []
for year in years:
    combine = draftcombinestats.DraftCombineStats(season_all_time = year)
    combine_df = combine.get_data_frames()
    
    combine_dfs.append(combine_df[0])
    sleep(0.6)
combine_data_df = pd.concat(combine_dfs)

combine_data_df.to_csv('combine_total.csv')         

KeyboardInterrupt: 

In [19]:
#time = 15:30 for .unique()

from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats
from nba_api.stats.endpoints import commonplayerinfo




def get_draft_info():
    
    draft_dfs = []
    
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'] in combine_data_df['PLAYER_NAME'].unique(): #try list() and .values()
            player_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id'])
            df = player_info.get_data_frames()
            draft_dfs.append(df[0])
            sleep(0.6)
            
    draft_data_df = pd.concat(draft_dfs)
    important_columns = ['PERSON_ID', 'DISPLAY_FIRST_LAST', 'SCHOOL', 'BIRTHDATE', 'SEASON_EXP', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER']
    
    draft_df = draft_data_df[important_columns]
    
    '''
    for player in combine_data_df['PLAYER_ID']:
        print(player)
        draft = commonplayerinfo.CommonPlayerInfo(player_id = player)
        draft_df = draft.get_data_frames()
        draft_dfs.append(draft_df[0])
        sleep(0.5)
    draft_data_df = pd.concat(draft_dfs)
    

     
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'] == name:
            player_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id'])
            df = player_info.get_data_frames()
            df[1]['SCHOOL'] = df[0]['SCHOOL']
    '''
    
    return(draft_df)
    
draft_df = get_draft_info()


In [20]:
columns_to_rename = {'DISPLAY_FIRST_LAST': 'Player'}
draft_df.rename(columns = columns_to_rename)
draft_df.to_csv('draft.csv')

In [21]:
list_of_columns = list(combine_data_df.columns)
columns_to_drop =   ['HEIGHT_W_SHOES',
                     'HEIGHT_W_SHOES_FT_IN',
                     'HEIGHT_WO_SHOES_FT_IN',
                     'WINGSPAN_FT_IN',
                     'BODY_FAT_PCT',
                     'HAND_LENGTH',
                     'HAND_WIDTH',
                     'MODIFIED_LANE_AGILITY_TIME',
                     'SPOT_FIFTEEN_CORNER_LEFT',
                     'SPOT_FIFTEEN_BREAK_LEFT',
                     'SPOT_FIFTEEN_TOP_KEY',
                     'SPOT_FIFTEEN_BREAK_RIGHT',
                     'SPOT_FIFTEEN_CORNER_RIGHT',
                     'SPOT_COLLEGE_CORNER_LEFT',
                     'SPOT_COLLEGE_BREAK_LEFT',
                     'SPOT_COLLEGE_TOP_KEY',
                     'SPOT_COLLEGE_BREAK_RIGHT',
                     'SPOT_COLLEGE_CORNER_RIGHT',
                     'SPOT_NBA_CORNER_LEFT',
                     'SPOT_NBA_BREAK_LEFT',
                     'SPOT_NBA_TOP_KEY',
                     'SPOT_NBA_BREAK_RIGHT',
                     'SPOT_NBA_CORNER_RIGHT',
                     'OFF_DRIB_FIFTEEN_BREAK_LEFT',
                     'OFF_DRIB_FIFTEEN_TOP_KEY',
                     'OFF_DRIB_FIFTEEN_BREAK_RIGHT',
                     'OFF_DRIB_COLLEGE_BREAK_LEFT',
                     'OFF_DRIB_COLLEGE_TOP_KEY',
                     'OFF_DRIB_COLLEGE_BREAK_RIGHT',
                     'ON_MOVE_FIFTEEN',
                     'ON_MOVE_COLLEGE',
                     'BENCH_PRESS'
                     ]                                       #2016 = none

combine_df = combine_data_df.drop(columns = columns_to_drop)
combine_df = combine_df.dropna()


columns_to_rename = {'PLAYER_NAME': 'Player'}
combine_df = combine_df.rename(columns = columns_to_rename)
combine_df.to_csv('combine.csv')

In [ ]:
#Scrapes around 200 webpages from basketball reference, Saves files in /NCAA
#Can run it if you would like but Selenium may crash every once in a while
#If this occurs I would suggest indexing list_of_urls to avoid scraping pages already scraped before the crash and re-running cell
'''
for url in list_of_urls:
    get_html_api(url)
'''
#All files nessecary already in /NCAA directory

In [19]:
#Goes through each HTML file in NCAA and extracts table as a df and adds it to the dfs_college list 
#If no table/player data in HTML, file is deleted
#Cell will take around 5 minutes to run
dfs_college = []
for url in list_of_urls:
    for year in years:
        file_name = generate_name(url, year)
        path = Path('./NCAA/{}.html'.format(file_name))
        if os.path.isfile(path):                                       #Checks if file exists
            with open('NCAA/{}.html'.format(file_name), encoding="utf-8") as f:
                page = f.read()
                soup = BeautifulSoup(page, 'html.parser')
                try:                                                   #Creates df from player data table and adds it to a list
                    soup.find('tr', class_= 'over_header').decompose() #This will raise AttributeError if there is no player data on webpage
                    table = soup.find(id=NCAA_id)                      
                    df = pd.read_html(str(table))[0]
                    df['Year'] = year
                    dfs_college.append(df)
                except AttributeError:                                 #If no player data table exists the HTML file is deleted
                    f.close()                                          #This occurs because some conferences did not exist until later 
                    os.remove(path)

In [20]:
#Adds all college players to a single df and gets relevant collumns
college_total = pd.concat(dfs_college)
relevant_columns = ['Player', 'Pos', 'G','MP','FG%','2P%','3P%','FT%','PTS.1','TRB.1','AST.1','STL.1','BLK.1','PER','WS','BPM', 'Year']
columns_to_rename = {'PTS.1': 'PTS',
                    'AST.1': 'AST',
                    'STL.1': 'STL',
                    'BLK.1': 'BLK',
                    'TRB.1': 'TRB'}

college_total_clean = college_total.loc[:,relevant_columns]
college_total_clean.rename(columns = columns_to_rename , inplace = True)

college_total_clean.to_csv('NCAA Stats.csv')

'''
#Merges df containing college player stats and NBA draft info to a single df which is written to total.csv for analysis
total = pd.merge(college_total_clean, combine_df, on=['Player', 'Year'])
total['3P%'] = total['3P%'].fillna(0)
total = total.dropna()
'''

"\n#Merges df containing college player stats and NBA draft info to a single df which is written to total.csv for analysis\ntotal = pd.merge(college_total_clean, combine_df, on=['Player', 'Year'])\ntotal['3P%'] = total['3P%'].fillna(0)\ntotal = total.dropna()\n"